In [25]:
from datasets import load_dataset
import markovify

dataset = load_dataset("biglam/gutenberg-poetry-corpus")
dataset['train'] = dataset['train'][:12000]

In [26]:
text_model = markovify.Text(dataset['train']['line'])
for i in range(5):
    print(text_model.make_sentence())

Drips the melting snow in drifts about it,
And he named her,
Felt the mighty Pau-Puk-Keewis
And the arrows of the red deer homeward,
Mean while in my dominions,


In [27]:
lines = []
for i in range(5):
    lines.append(text_model.make_sentence())

for line in lines:
    print(line)

Mov'd on in silence all the coasts of Light
Puk-Wudj'ies, little wild men of the North, there to dwell here, driv'n out from God
And in the trees with scarlet,
With envie against the winter,
Shall be the sun descending,


In [28]:
from textblob import TextBlob
import pandas as pd

def sentiment_analysis(poetry):
    def getSubjectivity(text):
        return TextBlob(text).sentiment.subjectivity

    def getPolarity(text):
        return TextBlob(text).sentiment.polarity

    poetry['TextBlob_Subjectivity'] = poetry['line'].apply(getSubjectivity)
    poetry['TextBlob_Polarity'] = poetry['line'].apply(getPolarity)

    def getAnalysis(score):
        if score < 0:
            return 'Negative'
        elif score == 0:
            return 'Neutral'
        else:
            return 'Positive'

    poetry['TextBlob_Analysis'] = poetry['TextBlob_Polarity'].apply(getAnalysis)
    
    return poetry

# Poem example
poem = {
    'line': lines
}

poetry_df = pd.DataFrame(poem)

result = sentiment_analysis(poetry_df)
print(result)


                                                line  TextBlob_Subjectivity  \
0        Mov'd on in silence all the coasts of Light                   0.70   
1  Puk-Wudj'ies, little wild men of the North, th...                   0.45   
2                     And in the trees with scarlet,                   0.00   
3                     With envie against the winter,                   0.00   
4                       Shall be the sun descending,                   0.00   

   TextBlob_Polarity TextBlob_Analysis  
0            0.40000          Positive  
1           -0.04375          Negative  
2            0.00000           Neutral  
3            0.00000           Neutral  
4            0.00000           Neutral  


In [29]:
import spacy
import numpy as np
from scipy.spatial.distance import cosine

# Încărcăm modelul spaCy
nlp = spacy.load("en_core_web_lg")

def find_synonym(word):
    # Obținem embedding-ul pentru cuvântul dat
    word_embedding = nlp(word).vector
    
    # Lista pentru a stoca distanțele și cuvintele corespunzătoare
    distances = []
    words = []
    
    # Iterăm prin toate cuvintele din vocabularul încărcat în spaCy
    for token in nlp.vocab:
        # Verificăm dacă cuvântul are un embedding și nu este cuvântul dat în sine
        if token.text != word and token.has_vector:
            # Calculăm similaritatea cosinusului între embedding-ul cuvântului dat și embedding-ul curent
            distance = cosine(word_embedding, token.vector)
            distances.append(distance)
            words.append(token.text)
    
    # Găsim cuvântul cel mai similar
    most_similar_word = words[np.argmin(distances)]
    
    return most_similar_word

In [30]:
def replace_with_sinonims(poem):
    new_poem = []
    for line in poem:
        new_vers = ''
        for word in line.split(' '):
            from numpy.random import randint
            p = randint(1,101)
            if p % 2 == 0:
                word = find_synonym(word)
            new_vers += word + ' '
        new_poem.append(new_vers)

    return new_poem

new_poem = replace_with_sinonims(lines)
for line in new_poem:
    print(line)

/home/codespace/.local/lib/python3.10/site-packages/scipy/spatial/distance.py:647: RuntimeWarning: invalid value encountered in divide
  dist = 1.0 - uv / math.sqrt(uu * vv)


ü. on where silence these the coasts and Might 
- somethin’ little those of and , there to dwell , driv'n here from lovin’ 
'Cause where and trees and scarlet, 
with space against the winter, 
Can be the sun , 


In [31]:
from nltk.translate.bleu_score import sentence_bleu

reference = [[word for word in line] for line in dataset['train']['line']]

candidate = [word for line in lines for word in line ]

score = sentence_bleu(reference, candidate)

print(score)

0.8603600215528934
